<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>

<br>

<a href="./341_ltpy_Ozone_hole_case_study.ipynb"><< 341 - 2019 Antarctic ozone hole case study </a><span style="float:right;"><a href="./index_ltpy.ipynb">Index >></a></span>

# 3.4.2  Ozone hole case study - CAMS animation

This notebook gives an example how the forecast data of the `Copernicus Atmosphere Monitoring Service (CAMS)` can be animated.

The example makes use of matplotlib's function `animation` and Jupyter notebooks function `HTML` to display HTML and video content within the notebook.

#### Load required libraries

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes

#### Load helper functions

In [3]:
from ipynb.fs.full.ltpy_functions import visualize_s5p_pcolormesh

<hr>

#### Load CAMS Near-real time for 10 September 2019

You can load the `CAMS Near-Real Time forecast` data with xarray's function `xr.open_dataset`. The example data has three dimensions, `latitude`, `longitude` and `time`. You can also see that the forecast data has 6 time steps, resulting into the initial run and 5 forecast days.

In [5]:
cams_fc = './eodata/cams/nrt/o3/2019/09/10/20190910_o3_fc.nc'
cams_fc_xr = xr.open_dataset(cams_fc)
cams_fc_xr

<xarray.Dataset>
Dimensions:    (latitude: 451, longitude: 900, time: 6)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.6 -179.2 ... 178.8 179.2 179.6
  * latitude   (latitude) float32 90.0 89.6 89.2 88.8 ... -89.2 -89.6 -90.0
  * time       (time) datetime64[ns] 2019-09-10 2019-09-11 ... 2019-09-15
Data variables:
    gtco3      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-12-08 10:15:35 GMT by grib_to_netcdf-2.14.1: grib_to_n...

#### Load the ozone data variable as `DataArray`

`gtco3` is the variable name you would like to load as `Data Array`. You see that the data is disseminated in `kg m**-2` as unit.

In [6]:
cams_o3 = cams_fc_xr['gtco3']
cams_o3

<xarray.DataArray 'gtco3' (time: 6, latitude: 451, longitude: 900)>
[2435400 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.6 -179.2 ... 178.8 179.2 179.6
  * latitude   (latitude) float32 90.0 89.6 89.2 88.8 ... -89.2 -89.6 -90.0
  * time       (time) datetime64[ns] 2019-09-10 2019-09-11 ... 2019-09-15
Attributes:
    units:      kg m**-2
    long_name:  GEMS Total column ozone

#### Convert from `kg m**-2` to `Dobson Unit`

Ozone data are often represented in `Dobson Unit`. Thus, with the factor `2.1415*1e-5`, you can easily convert from `kg m**-2` to `Dobson Unit`.

In [7]:
cams_o3_du = cams_o3 / (2.1415*1e-5)
cams_o3_du

<xarray.DataArray 'gtco3' (time: 6, latitude: 451, longitude: 900)>
array([[[282.82578, 282.82578, ..., 282.82578, 282.82578],
        [283.98798, 283.99817, ..., 283.96756, 283.97778],
        ...,
        [232.94267, 232.94775, ..., 232.92227, 232.93246],
        [230.96997, 230.96997, ..., 230.96997, 230.96997]],

       [[283.4986 , 283.4986 , ..., 283.4986 , 283.4986 ],
        [286.2665 , 286.2818 , ..., 286.2461 , 286.25632],
        ...,
        [228.3448 , 228.32953, ..., 228.3703 , 228.35501],
        [227.63628, 227.63628, ..., 227.63628, 227.63628]],

       ...,

       [[274.78717, 274.78717, ..., 274.78717, 274.78717],
        [276.36737, 276.36737, ..., 276.37247, 276.37247],
        ...,
        [255.59549, 255.60057, ..., 255.58528, 255.5904 ],
        [253.86237, 253.86237, ..., 253.86237, 253.86237]],

       [[269.94974, 269.94974, ..., 269.94974, 269.94974],
        [271.09668, 271.09668, ..., 271.09155, 271.09668],
        ...,
        [251.17093, 251.16075, ...,

#### Animate the total column ozone over the 5 day forecast

The animation function consists of 4 parts:
- **Setting the initial state:**<br>
 Here, you define the general plot your animation shall use to initialise the animation. You can also define the number of frames (time steps) your animation shall have
 
 
- **Functions to animate:**<br>
 An animation consists of three functions: `draw()`, `init()` and `animate()`. `Draw()` is the function where you individual frames are passed on and the figure is returned as image. In this example, the function redraws the plot for each time step. `init()` returns the figure you defined right at the beginning. `animate()` returns the `draw()` function.
 
 
- **Create a `animate.FuncAnimation` object:** <br>
 The functions defined before are now combined to build an `animate.FuncAnimation` object.
 
 
- **Play the animation as video:**<br>
 As a final step, you can integrate the animation into the notebook with the `HTML` class. You take the generate animation object and convert it to a HTML5 video with the `to_html5_video` function

In [19]:
# Setting the initial state:
# 1. Define figure for initial plot
fig, ax = visualize_s5p_pcolormesh(cams_o3_du[0,:,:],cams_o3_du.longitude, cams_o3_du.latitude,ccrs.Orthographic(180,-90), 
                        'jet', 'DU',cams_o3.long_name + ' '+ str(cams_o3_du.time[0].data), 100,600, -180, 179.6, -90. , 90., log=False,set_global=True)

frames = 6

def draw(i):
    img = plt.pcolormesh(cams_o3_du.longitude, cams_o3_du.latitude, cams_o3_du[i,:,:], 
                        cmap='jet', transform=ccrs.PlateCarree(),
                        vmin=100,
                        vmax=600)
    ax.set_title(cams_o3.long_name + ' '+ str(cams_o3_du.time[i].data), fontsize=20, pad=20.0)
    return img


def init():
    return fig


def animate(i):
    return draw(i)

ani = animation.FuncAnimation(fig, animate, frames, interval=800, blit=False,
                              init_func=init, repeat=True)

HTML(ani.to_html5_video())
plt.close(fig)


#### Play the animation as HTML5 video

In [20]:
HTML(ani.to_html5_video())

<br>

<a href="./341_ltpy_Ozone_hole_case_study.ipynb"><< 341 - 2019 Antarctic ozone hole case study </a><span style="float:right;"><a href="./index_ltpy.ipynb">Index >></a></span>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eo-lab/training-atmospheric-composition/">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>